In [48]:
from moku.instruments import MultiInstrument, PIDController, FrequencyResponseAnalyzer
import control as ct
import altair as alt
import pandas as pd
import numpy as np

In [49]:
def PISDS_controller(prop_gain_dB,int_crossover,int_saturation_dB,diff_crossover,diff_saturation_dB):
    diff_saturation_mag = 10**(diff_saturation_dB/20)
    int_saturation_mag = 10**(int_saturation_dB/20)
    prop_gain_mag = 10**(prop_gain_dB/20)
    pid = ct.tf([1,diff_crossover,int_crossover*diff_crossover],[diff_crossover/prop_gain_mag,0])
    sat = ct.zpk([0],[-diff_crossover*diff_saturation_mag/prop_gain_mag,prop_gain_mag*int_crossover/int_saturation_mag],diff_crossover*diff_saturation_mag/prop_gain_mag)
    return ct.series(pid,sat)


In [50]:
#controller parameters:
prop_gain_dB=-30
int_crossover = 5e3
diff_crossover = 6e3
int_saturation_dB = 0
diff_saturation_dB = 20

In [51]:
#create model transfer function
model_TF = PISDS_controller(prop_gain_dB,int_crossover,int_saturation_dB,diff_crossover,diff_saturation_dB)

In [52]:
#Configure the MOKU to set and measure a PID controller with these parameters
MIM = MultiInstrument('192.168.50.97', force_connect=True, platform_id=4) #192.168.50.97 is the IP for Moku Pro 1

pid = MIM.set_instrument(1, PIDController)
fra = MIM.set_instrument(2, FrequencyResponseAnalyzer)

connections = [dict(source="Slot1OutA", destination="Slot2InA"),
               dict(source="Slot2OutA", destination="Slot1InA")]

MIM.set_connections(connections=connections)

[{'destination': 'Slot1InA', 'source': 'Slot2OutA'},
 {'destination': 'Slot2InA', 'source': 'Slot1OutA'},
 {'destination': 'Slot2InB', 'source': 'Slot1OutB'}]

In [53]:
#Set the PID controller
pid.set_by_frequency(channel=1, prop_gain=prop_gain_dB,int_crossover = int_crossover,diff_crossover = diff_crossover,int_saturation = int_saturation_dB,diff_saturation = diff_saturation_dB)
pid.enable_input(1, True)
pid.enable_output(1,True,True)

{'output': True, 'signal': True}

In [54]:
fra.set_sweep(start_frequency=10, stop_frequency=20e6, num_points=256,
                averaging_time=1e-3, averaging_cycles=1, settling_cycles=1,
                settling_time=1e-3)
fra.set_output(1, 0.01)

{'amplitude': 0.01, 'offset': 0.0}

In [55]:
delay = fra.start_sweep() 
print(delay)
data = fra.get_data(wait_complete = True)

{'estimated_sweep_time': 3.9502599500690194}


In [56]:

MIM.relinquish_ownership()

In [62]:
df = pd.DataFrame(data = data['ch1'])
df = df.drop(columns = 'phase')

omega = 2*np.pi*df.frequency

df

,frequency,magnitude
0,9.999992e+00,0.196630
1,1.058546e+01,0.194557
2,1.120520e+01,0.192173
3,1.186122e+01,0.189663
4,1.255565e+01,0.186652
...,...,...
251,1.592907e+07,20.297725
252,1.686166e+07,20.260090
253,1.784885e+07,20.217929
254,1.889383e+07,20.175762


In [63]:
(model_mag ,model_phase, omega) =  model_TF.freqresp(omega = omega)
model_mag_dB = 20*np.log10(model_mag)
df.insert(2,"model mag",model_mag_dB)
print(df)

        frequency  magnitude  model mag
0    9.999992e+00   0.196630  -0.637218
1    1.058546e+01   0.194557  -0.708075
2    1.120520e+01   0.192173  -0.786124
3    1.186122e+01   0.189663  -0.871947
4    1.255565e+01   0.186652  -0.966144
..            ...        ...        ...
251  1.592907e+07  20.297725  19.998439
252  1.686166e+07  20.260090  19.998607
253  1.784885e+07  20.217929  19.998757
254  1.889383e+07  20.175762  19.998891
255  2.000000e+07  20.118567  19.999010

[256 rows x 3 columns]


In [67]:
base = alt.Chart(df).encode(x = alt.X('frequency:Q').scale(type = "log"))

#alt.layer(
    #base.mark_line(color='blue').encode(y='magnitude:Q'),
    #base.mark_line(color='red').encode(y='model mag:Q')
#)

base

SchemaValidationError: '{'data': {'name': 'data-f0af4af047ee7d82ddee99c534ad013a'}, 'encoding': {'x': {'field': 'frequency', 'scale': {'type': 'log'}, 'type': 'quantitative'}}}' is an invalid value.

'mark' is a required property

alt.Chart(...)